In [ ]:
# Use if you run the notebook on Google colab
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

In [ ]:
!pip install mglearn

# 3: Linear Regression

## Imports

In [ ]:
import os
import sys

sys.path.append("/content/drive/MyDrive/50603/code")
os.chdir('/content/drive/MyDrive/50603')

import IPython
import ipywidgets as widgets
import matplotlib.pyplot as plt
import mglearn
import numpy as np
import pandas as pd
from IPython.display import Image, HTML, display
from ipywidgets import interact, interactive
from plotting_functions import *
from sklearn.dummy import DummyClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.impute import SimpleImputer
from sklearn.model_selection import cross_val_score, cross_validate, train_test_split
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from utils import *

%matplotlib inline
pd.set_option("display.max_colwidth", 200)

## Learning outcomes

From this lecture, students are expected to be able to:

- Explain the general intuition behind linear models;
- Explain how `predict` works for linear regression;
- Use `scikit-learn`'s `Ridge` model;
- Demonstrate how the `alpha` hyperparameter of `Ridge` is related to the fundamental tradeoff;

<br><br>

### Linear regression

- A very popular statistical model and has a long history.  
- Imagine a hypothetical regression problem of predicting weight of a snake given its length.

In [ ]:
np.random.seed(7)
n = 100
X_1 = np.linspace(0, 2, n) + np.random.randn(n) * 0.01
X = pd.DataFrame(X_1[:, None], columns=["length"])

y = abs(np.random.randn(n, 1)) * 3 + X_1[:, None] * 5 + 0.2
y = pd.DataFrame(y, columns=["weight"])
snakes_df = pd.concat([X, y], axis=1)
train_df, test_df = train_test_split(snakes_df, test_size=0.2, random_state=77)

X_train = train_df[["length"]]
y_train = train_df["weight"]
X_test = test_df[["length"]]
y_test = test_df["weight"]
train_df.head()

Let's visualize the hypothetical snake data.

In [ ]:
plt.plot(X_train.to_numpy(), y_train, ".", markersize=10)
plt.xlabel("length")
plt.ylabel("weight (target)");

Let's plot a linear regression model on this dataset.

In [ ]:
grid = np.linspace(X_train.min()[0], X_train.max()[0], 1000)
grid = grid.reshape(-1, 1)

In [ ]:
from sklearn.linear_model import Ridge

r = Ridge()
r.fit(X_train.to_numpy(), y_train)
plt.plot(X_train.to_numpy(), y_train, ".", markersize=10)
plt.plot(grid, r.predict(grid))
plt.grid(True)
plt.xlabel("length")
plt.ylabel("weight (target)");

**The orange line is the learned linear model.**

### Prediction of linear regression

- Given a snake length, we can use the model above to predict the target (i.e., the weight of the snake).
- The prediction will be the corresponding weight on the orange line.

In [ ]:
snake_length = 0.75
r.predict([[snake_length]])

#### What are we exactly learning?

- The model above is a **line**, which can be represented with a **slope** (i.e., coefficient or weight) and an **intercept**.
- For the above model, we can access the slope (i.e., coefficient or weight) and the intercept using `coef_` and `intercept_`, respectively.

In [ ]:
r.coef_  # r is our linear regression object

In [ ]:
r.intercept_  # r is our linear regression object

### How are we making predictions?
- Given a feature value $x_1$ and learned coefficient $w_1$ and intercept $b$, we can get the prediction $\hat{y}$ with the following formula:
$$\hat{y} = w_1x_1 + b$$

In [ ]:
prediction = snake_length * r.coef_ + r.intercept_
prediction

In [ ]:
r.predict([[snake_length]])

Great! Now we exactly know how the model is making the prediction.

### Generalizing to more features
For more features, the model is a higher dimensional hyperplane and the general prediction formula looks as follows:

$\hat{y} =$ <font color="red">$w_1$</font> <font color="blue">$x_1$ </font> $+ \dots +$ <font color="red">$w_d$</font> <font color="blue">$x_d$</font> + <font  color="green"> $b$</font>

where,
- <font  color="blue"> ($x_1, \dots, x_d$) are input features </font>
- <font  color="red"> ($w_1, \dots, w_d$) are coefficients or weights </font> (learned from the data)
- <font  color="green"> $b$ is the bias which can be used to offset your hyperplane </font> (learned from the data)

### Example

- Suppose these are the coefficients learned by a linear regression model on a hypothetical housing price prediction dataset.

| Feature | Learned coefficient |
|--------------------|---------------------:|
| Bedrooms | 0.20 |
| Bathrooms| 0.11 |
| Square Footage | 0.002 |
| Age | -0.02 |

- Now given a new example, the target will be predicted as follows:
| Bedrooms | Bathrooms | Square Footage | Age |
|--------------------|---------------------|----------------|-----|
| 3                  | 2                   | 1875           | 66  |

$$\hat{y} = w_1x_1 + w_2x_2 + w_3x_3 + w_4x_4 + b$$

$$\text{predicted price}=  0.20 \times 3 + 0.11 \times 2 + 0.002 \times 1875 + (-0.02) \times 66 + b$$

When we call `fit`, a coefficient or weight is learned for each feature which tells us the role of that feature in prediction. These coefficients are learned from the training data.  

***Note***
> In linear models for regression, the model is a line for a single feature, a plane for two features, and a hyperplane for higher dimensions. We are not yet ready to discuss how does linear regression learn these coefficients and intercept.

### `Ridge`

- `scikit-learn` has a model called `LinearRegression` for linear regression.
- But if we use this "vanilla" version of linear regression, it may result in large coefficients and unexpected results.
- So instead of using `LinearRegression`, we will _always use another linear model called `Ridge`_, which is a linear regression model with a complexity **hyperparameter** `alpha`.

In [ ]:
from sklearn.linear_model import LinearRegression  # DO NOT USE IT IN THIS COURSE
from sklearn.linear_model import Ridge  # USE THIS INSTEAD

#### Data

We will use a dataset of residential homes in Ames, Iowa. Using the non-categorical data in this dataset, we attempt to **predict the final price** of each home.

In [ ]:
# The Ames housing dataset
from sklearn.datasets import fetch_openml
ames = fetch_openml(name="house_prices", as_frame=True)

To simplify our demonstration here, we will keep most of *numerical columns* and drop the rest of (categorical) columns. This simplifies column preprocessing and our upcoming analysis in this exercise.

In [ ]:
ames_df = pd.DataFrame(ames.data, columns=ames.feature_names).set_index('Id')
ames_df = ames_df.select_dtypes('number') # drop non-numerical cols, see note above
ames_df = ames_df.drop(columns=['MSSubClass', 'MoSold', 'YrSold']) # drop more cols, see note above

X_train, X_test, y_train, y_test = train_test_split(
    ames_df, ames.target, test_size=0.2
)

[d.shape for d in [X_train, X_test, y_train, y_test]] # see the train and test sizes

An extended description of the Ames housing dataset is accessible through `ames.url` and `ames.DESCR`:

In [ ]:
print("Ames housing dataset link:", ames.url)
print("Ames housing dataset description:\n\n", ames.DESCR[:480])  # print only the introduction
# print("Ames housing dataset description:\n\n", ames.DESCR)  # print the full description

#### `Ridge` on the Ames housing dataset

In [ ]:
preprocess = make_pipeline(SimpleImputer(), StandardScaler())
pipe = make_pipeline(preprocess, Ridge())
scores = cross_validate(pipe, X_train, y_train, return_train_score=True)
print(pd.DataFrame(scores).mean().rename('mean').to_frame().T)
pd.DataFrame(scores)

#### Hyperparameter `alpha` of `Ridge`

- Ridge has hyperparameters just like the rest of the models we learned.
- The alpha hyperparameter is what makes `Ridge` different from vanilla `LinearRegression`.
- Similar to the other hyperparameters that we saw, `alpha` controls the fundamental tradeoff.

***Note***
> If we set `alpha=0` that is the same as using LinearRegression.

Let's examine the effect of `alpha` on the fundamental tradeoff.

In [ ]:
scores_dict = {
    "alpha": 10.0 ** np.arange(-2, 6, 1),
    "mean_train_scores": list(),
    "mean_cv_scores": list(),
}
for alpha in scores_dict["alpha"]:
    pipe_ridge = make_pipeline(preprocess, Ridge(alpha=alpha))
    scores = cross_validate(pipe_ridge, X_train, y_train, return_train_score=True)
    scores_dict["mean_train_scores"].append(1-scores["train_score"].mean())
    scores_dict["mean_cv_scores"].append(1-scores["test_score"].mean())

results_df = pd.DataFrame(scores_dict)

In [ ]:
results_df

In [ ]:
# Plot only the top part of the table for better viewing
ax = results_df.head(6).set_index('alpha').plot()
ax.invert_xaxis()

Thus, `alpha = 100` is the optimum value here. In general,
- larger `alpha` $\rightarrow$ likely to underfit
- smaller `alpha` $\rightarrow$ likely to overfit

#### Coefficients and intercept

The model learns
- coefficients associated with each feature
- the intercept or bias

Let's examine the coefficients learned by the model.

In [ ]:
pipe_ridge = make_pipeline(preprocess, Ridge(alpha=1.0))
pipe_ridge.fit(X_train, y_train)
coeffs = pipe_ridge.named_steps["ridge"].coef_

In [ ]:
pd.DataFrame(data=coeffs, index=ames_df.columns, columns=["Coefficients"])

***Important***
> Take these coefficients with a grain of salt. They might not always match your intuitions.

- The model also learns an intercept (bias).
- For each prediction, we are adding this amount irrespective of the feature values.  

In [ ]:
pipe_ridge.named_steps["ridge"].intercept_

Can we use this information to interpret model predictions?

## Interpretation of coefficients

- One of the main advantages of linear models is that they are relatively **easy to interpret**.
- We have **one coefficient per feature** which kind of describes the **role of the feature** in the prediction according to the model.

There are two pieces of information in the coefficients based on

- Sign
- Magnitude

### Sign of the coefficients

In the example below, for instance:
- `OverallQual` (Rates the overall material and finish of the house) has a **positive coefficient**
    - the prediction will be **proportional** to the feature value; as `OverallQual` gets bigger, the median house value gets bigger
- `LowQualFinSF`: (Low quality finished square feet, all floors) has a **negative coefficient**
    - the prediction will be **inversely proportional** to the feature value; as `LowQualFinSF` gets bigger, the median house value gets smaller

In [ ]:
pd.DataFrame(
    data=coeffs, index=ames_df.columns, columns=["Coefficients"]
).loc[['OverallQual', 'LowQualFinSF']]

#### Magnitude of the coefficients

- Bigger magnitude $\rightarrow$ bigger impact on the prediction
- In the example below, both `OverallQual` and `Fireplaces` have a positive impact on the prediction but `OverallQual` would have a bigger positive impact because it's feature value is going to be multiplied by a number with a bigger magnitude.
- Similarly both `LowQualFinSF` and `BsmtUnfSF` have a negative impact on the prediction but `LowQualFinSF` would have a bigger negative impact because it's going to be multiplied by a number with a bigger magnitude.

In [ ]:
data = {
    "coefficient": pipe_ridge.named_steps["ridge"].coef_.tolist(),
    "magnitude": np.absolute(pipe_ridge.named_steps["ridge"].coef_.tolist()),
}
coef_df = pd.DataFrame(data, index=ames_df.columns).sort_values(
    "magnitude", ascending=False
)
coef_df.loc[['OverallQual', 'Fireplaces', 'LowQualFinSF', 'BsmtUnfSF']]

### Importance of scaling
- When you are interpreting the model coefficients, scaling is crucial.
- If you do not scale the data, features with smaller magnitude are going to get coefficients with bigger magnitude whereas features with bigger scale are going to get coefficients with smaller magnitude.
- That said, when you scale the data, feature values become hard to interpret for humans!

***Important***
> Take these coefficients with a grain of salt. They might not always match your intuitions. Also, they do not tell us about how the world works. They only tell us about how the prediction of your model works.

<br><br>